In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import numpy as np
import pandas as pd
import re



In [3]:
files = !gsutil ls gs://ds-url-catag/Stickers/sticker_categories/*/

In [ ]:
count = 0 
remove_quotes = udf(lambda x: re.sub('"','',x), StringType())
remove_slashes = udf(lambda x: re.sub('\\\\','',x), StringType())
for sub_file in files:
    sub_str = sub_file.split("/")[6]
    extract_string = sub_str[13:]   
    category = re.sub(r'[0-9]','',extract_string)
    if sub_str != '':
        try :
            df = sc.textFile("gs://ds-url-catag/Stickers/sticker_categories/*/"+sub_str+"/").map(lambda line: line.split(",")).map(lambda line: (line[0],line[1],line[2],line[3])).toDF(['Sticker_id','name','animated','Type'])
            df = df.withColumn('category',lit(category))
            df = df.select(*[remove_quotes(column).alias(column) for column in df.columns])
            df = df.select(*[remove_slashes(column).alias(column) for column in df.columns])
            df.write.parquet("gs://ds-url-catag/Stickers/processed_categories/"+sub_str+"/")
        except ValueError:
            count += 1
        
        
        